# Compiling NIFTI files from tiff stack files

For the compatibility in the following ANTs registration,  
the images taken from the other side to the main angle are flipped and re-oriented.  

You can interactively set the annotation phrase.  
For example, if there are files like the following:  
- descSPIM_angle0_GFP_target_signal_some_notes.tif
- descSPIM_angle180_GFP_target_signal_some_notes.tif
- descSPIM_angle0_PI_nuclear_stain_some_notes.tif
- descSPIM_angle180_PI_nuclear_stain_some_notes.tif  

**To set the 'angle0' as the main angle, enter in the message box 'angle180'**.  
All the images that contains 'angle2' will be flipped both horizontally and along z axis, so that the compiled NIFTI files can be used in the following ANTs registration steps.

In [ ]:
%%time

import os
import re
import glob
import tifffile
import numpy as np
import nibabel as nb
from skimage.transform import rescale
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor

def load_tiff_stack(imgpath):
    stack = tifffile.imread(imgpath)
    return stack

def flip_tiff_stack(stack):
    return np.flip(np.flip(stack, axis=2), axis=0)

def normalize_and_scale(img):
    img = (img - img.min()) / (img.max() - img.min())
    img = (img * 255).astype(np.uint8)
    return img

def save_nifti(stack, niftiname, spx, spy, spz):
    stack = np.swapaxes(stack,0,2)
    nim = nb.Nifti1Image(stack, affine=None)
    aff = np.diag([-spx,-spy,spz,1])
    nim.header.set_qform(aff, code=2)
    nim.to_filename(niftiname)

def process_folder(base_dir, filename, flip_key, spx, spy, spz):
    file_name = os.path.splitext(os.path.basename(filename))[0]

    full_stack = load_tiff_stack(filename)
    if flip_key in file_name:
        file_name = file_name.replace(flip_key, f'Flipped{flip_key}')
        full_stack = flip_tiff_stack(full_stack)
    # Save full resolution nifti
    save_nifti(full_stack, os.path.join(base_dir, f'{file_name}_fullsize.nii.gz'), spx, spy, spz)
    # Save downsize nifti
    down_stack = full_stack[::4, :, :]
    down_stack = rescale(down_stack, (1, 0.5, 0.5), anti_aliasing=True, channel_axis=None)
    #down_stack = rescale(down_stack, (1, 0.25, 0.25), anti_aliasing=True, channel_axis=None) # for 25% downsize
    down_stack = normalize_and_scale(down_stack)

    save_nifti(down_stack, os.path.join(base_dir, f'{file_name}_downsize.nii.gz'), spx*2, spy*2, spz*4)
    #save_nifti(down_stack, os.path.join(base_dir, f'{file_name}_downsize.nii.gz'), spx*4, spy*4, spz*4) #for 25% downsize
    
def extract_prefix(s):
    # Extract numbers
    number_part = re.search(r'\d+', s)
    if number_part:
        number_part = number_part.group()
    else:
        return None, s

    # Remove numbers from original string
    string_part = re.sub(r'\d+', '', s)

    return string_part


if __name__ == "__main__":
    # base directory the images are located
    base_path = input("Please enter base folder path: ")
    # if you use descSPIM-basic, the pixel size for (x,y)=(3.45,3.45)
    spx=float(3.45)
    spy=float(3.45)
    # minimum 5um for FA mode, 10um for FF mode
    spz = float(input('Please enter the pixel size for z axis:'))
    #spz=float(10)
    #spz=float(5)

    flip_key = input("Enter the annotation for the angle180 (e.g.;angle180, Angle180, angle2 etc.) :")
    main_key = f'{extract_prefix(flip_key)}0'

    file_list = glob.glob(os.path.join(base_path, "*.tif"))

    with ProcessPoolExecutor(max_workers=mp.cpu_count()) as executor:
        executor.map(process_folder, [base_path]*len(file_list), file_list, [flip_key]*len(file_list), [spx]*len(file_list), [spy]*len(file_list), [spz]*len(file_list))
    print('Compiling nifti successfully finished.')

# Before Using This Code
## 1. Dependencies
This script relies on several Python packages as well as external tools. Ensure you have the following dependencies installed on your system:

Python packages: nibabel, numpy, subprocess, os, multiprocessing, datetime
External tool: ANTs (Advanced Normalization Tools)
You can install the required Python packages using pip:`pip install nibabel numpy`


## 2. Install ANTs
ANTs is a powerful tool that allows us to perform complex imaging tasks, such as image registration, that go beyond what's possible with standard Python packages. 

### 2-1. Easy build
You can use this [bash script](https://github.com/dbsb-juntendo/descSPIM/blob/main/DOCs/codes/installants.sh) that downloads and installs ANTs in a unix environment with Bash shell. To use this script:  
#### Download the Script
Note that ANTs will be built in the same directory as this executable file is located, so you should save the file in the directory you want to build ANTs.
#### Set Permissions
You'll need to make the script executable. In your terminal, navigate to the directory containing the script, and run:  
`chmod +x installANTs.sh`
#### Run the Script
Now you can run the script using:  
`./installANTs.sh`  
The script will automatically download ANTs, build it, and install it in a directory called install under the current directory.
#### Set Environment Variables
After the script runs successfully, it will export ANTSPATH and PATH variables for the current session. However, these exports won't persist across sessions. For this, the script will add these export commands to your ~/.bashrc file (if you're using Bash).  
To immediately apply these changes to your current terminal session, run:  
`source ~/.bashrc`  
If you're using a different shell or don't want to modify your ~/.bashrc, you'll need to manually add the appropriate export commands to your shell's configuration file or environment.  


### 2-2. Build on your own
#### Download and install
ANTs can be downloaded and installed from the official GitHub repository: https://github.com/ANTsX/ANTs. Follow the instructions there to build and install ANTs on your system.  
For MAC and Linux users: https://github.com/ANTsX/ANTs/wiki/Compiling-ANTs-on-Linux-and-Mac-OS  
For Windows users: https://github.com/ANTsX/ANTs/wiki/Compiling-ANTs-on-Windows-10  


#### Set ANTs Path
After installing ANTs, you need to add it to your system's PATH so that your script can access the ANTs tools. This can be done by modifying your .bashrc file: Open a terminal and type `nano ~/.bashrc` to open your .bashrc file in a text editor.

At the end of the file, add the following lines:  
`export ANTSPATH=/path/to/ANTs/install/bin && export PATH=${ANTSPATH}:$PATH`  

Replace /path/to/ANTs/install/bin with the path where you installed ANTs.Usually, the ANTs install path is shown at the end of the intalling log(standard outputs).

Press CTRL+O to save the file, then CTRL+X to exit the text editor.

In the terminal, type `source ~/.bashrc` to apply the changes.  


## 3. Verify ANTs Installation
To verify that ANTs is installed correctly and can be accessed by your script, 
type `which antsRegistration` in the terminal. This should return the path to the antsRegistration executable.

If the which command doesn't return anything, or you get an error, it means that the ANTs tools are not on your system's PATH. In this case, go back to step 3 and make sure you added the correct path to your .bashrc file.

Note: These instructions assume a Linux environment and a bash shell. If you are using a different system or shell, you might need to adjust these instructions accordingly.

***

# Processes
This code generate the registered then fused image both for the,  
- Nuclear stain channel
- Target channel  

So, **You need to prepare the eight .nii.gz files in total.**  
For both the fixed FOV(field of view) and the corresponding FOV (2x),  
there are both the nuclear stain and the target channel (2x),  
need downsized and fullsize images (2x) for each.  

Utilizing this code, generating these nifti files have been already done.
Just follow the instructions after running the next cell.


## 1.Target channel
For the Target channel, e.g., Thy1-GFP, C-fos etc.  
**1-1. Pre-registration:**
Get Affine transform ANTs matrix using downsized Nuclear Stain images.  
**1-2. Apply affine transform:**
Perform the transform to the downsized target channel image to be used as a moving image.  
**1-3. SyN Registration:**
Get Affine and SyN transform ANTs matrix using the obtained image and the target channel reference image in the target channel downsized images.  
**1-4. Apply transforms sequentially:**
Apply the affine transform which is obtained in 1-1 and the transforms of 1-3 to the fullsize image of the target channel.  
**(Optional) Convert to 8bit:**
**Highly recommended to save the disc space.** Convert the output of the 1-4 to 8-bit which is originally 32-bit.  
**1-5. Fuse:**
Average fusing the fullsize fixed and moved image.
The image is saved to the same folder as the input files are included, named as "TargetCh_Average_fused_fullsize.nii.gz".  

## 2.Nuclear stain channel
**2-1. SyN Registration:**
Get Affine and SyN transform ANTs matrix using the downsized images of nuclear stain channel.  
**2-2. Apply transform:**
Apply the transforms to the fullsize moving image.  
**(Optional) Convert to 8bit:**
**Highly recommended to save the disc space.** Convert the output of the 1-4 to 8-bit which is originally 32-bit.  
**2-3. Fuse:**
Average fusing the fullsize fixed and moved image.
The image is saved to the same folder as the input files are included, named as "NuclearStainCh_Average_fused_fullsize.nii.gz".

In [ ]:
%%time

import subprocess
import os
import datetime
import shutil
import time
import nibabel as nib
import numpy as np
import multiprocessing
import glob

# Set th number of threads to be used
os.environ["ITK_GLOBAL_DEFAULT_NUMBER_OF_THREADS"] = str(multiprocessing.cpu_count())
# If you want to use partially use the threads, comment out above(add # to in the front) and use below:
#os.environ["ITK_GLOBAL_DEFAULT_NUMBER_OF_THREADS"] = "100"

# Set paths
#base_path = input("Enter the parent folder name:")
nuclearstain_key = input("Enter the Nuclear-stain channel keywod:")
target_key = input("Enter the Target channel keywod:")
file_list = glob.glob(os.path.join(base_path, "*.nii.gz"))
for file in file_list:
    file_name = os.path.splitext(os.path.basename(file))[0]
    if nuclearstain_key in file_name and main_key in file_name and "downsize" in file_name:
        fixed_image = file
        print(f'fixed_image:{file}')
    elif nuclearstain_key in file_name and flip_key in file_name and "downsize" in file_name:
        moving_image = file
        print(f'moving_image:{file}')
    elif target_key in file_name and main_key in file_name and "downsize" in file_name:
        reference_image_downsize = file
        print(f'reference_image_downsize:{file}')
    elif target_key in file_name and flip_key in file_name and "downsize" in file_name:
        input_image_downsize = file
        print(f'input_image_downsize:{file}')
    elif target_key in file_name and main_key in file_name and "fullsize" in file_name:
        reference_image_fullsize = file
        print(f'reference_image_fullsize:{file}')
    elif target_key in file_name and flip_key in file_name and "fullsize" in file_name:
        input_image_fullsize = file
        print(f'input_image_fullsize:{file}')
    elif nuclearstain_key in file_name and main_key in file_name and "fullsize" in file_name:
        reference_fullsize = file
        print(f'reference_fullsize:{file}')
    elif nuclearstain_key in file_name and flip_key in file_name and "fullsize" in file_name:
        input_fullsize = file
        print(f'input_fullsize:{file}')

# Make the saving folders
date = datetime.date.today()
# Format as MMDD
date= date.strftime("%m%d")

new_directory = os.path.join(base_path, f"ants_io_TargetCh_{date}")
os.makedirs(new_directory, exist_ok=True)
new_path = os.path.join(base_path, f"ants_io_NuclearStainCh_{date}")
os.makedirs(new_path, exist_ok=True)

###
# Functions
###

# Registration function
def ants_registration(ants_cmd, log_path, new_directory=new_directory):
    # Run the ANTs registration command, capture the output in real-time and save it to the log file
    with open(log_path, "w") as log_file:
        process = subprocess.Popen(ants_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

        # Read the output line by line and print it to the console, while also saving it to the log file
        for line in iter(process.stdout.readline, ""):
            print(line, end="")
            log_file.write(line)

        stdout, stderr = process.communicate()
        process.stdout.close()
        return_code = process.wait()

    # Check the return code to see if the command was successful
    if return_code == 0:
        print("ANTS registration succeeded.")
    else:
        print("ANTS registration failed.")

#  Applying transform function       
def ants_apply_transform(ants_apply_cmd, log_path, reference_image, input_image, transformation_matrix):
    # Write the additional information to the log file
    with open(log_path, "w") as log_file:
        log_file.write(f"Reference Image: {reference_image}\n")
        log_file.write(f"Input Image: {input_image}\n")
        log_file.write(f"Transformation Matrix: {transformation_matrix}\n")
        log_file.write(f"antsApplyTransforms Command: {' '.join(ants_apply_cmd)}\n\n")

    # Get the start time
    start_time = time.time()

    # Run the antsApplyTransforms command, capture the output in real-time and add it to the log file
    with open(log_path, "a") as log_file:  # Use "a" to append to the file
        process = subprocess.Popen(ants_apply_cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

        # Read the output line by line and print it to the console, while also saving it to the log file
        for line in iter(process.stdout.readline, ""):
            print(line, end="")
            log_file.write(line)

        stdout, stderr = process.communicate()
        process.stdout.close()
        return_code = process.wait()
        
    # Get the end time and calculate the elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Write the elapsed time to the log file
    with open(log_path, "a") as log_file:
        log_file.write(f"\nElapsed time: {elapsed_time} seconds\n")

    # Check the return code to see if the command was successful
    if return_code == 0:
        print("Transform applied successfully.")
    else:
        print("Error applying transform.")

        
# Converting images to a 8-bit, because ANTs genrates a 32-bit(or 64-bit if you use 'double') images
# Delete the original 32(or64)-bit image to save the disc space. If you want to retain the original one, 
# please erase or comment-out the last line of this function
def Convert8bit(pre_path, converted_path):
    # Load the 32-bit compressed NIfTI file
    nifti_file = nib.load(pre_path)
    # Get the data from the NIfTI file
    data = nifti_file.get_fdata()
    # Normalize the data to 0-255
    data_normalized = ((data - np.min(data)) / (np.max(data) - np.min(data))) * 255
    # Convert to 8-bit
    data_8bit = data_normalized.astype(np.uint8)
    # Create a new NIfTI image with the 8-bit data and the original header
    nifti_8bit = nib.Nifti1Image(data_8bit, nifti_file.affine, nifti_file.header.copy())
    # Update the datatype in the header
    nifti_8bit.header.set_data_dtype(np.uint8)
    # Reset some fields to ensure they are valid for the new 8-bit data
    nifti_8bit.header['cal_min'] = 0
    nifti_8bit.header['cal_max'] = 255
    nifti_8bit.header['scl_slope'] = 1
    nifti_8bit.header['scl_inter'] = 0
    # Save the 8-bit compressed Nifti file with a new name
    nib.save(nifti_8bit, converted_path)
    
    # Delete the original file to save the disc space
    os.remove(pre_path)

    
# Fusing function
# Normaly, use 'average'. 
def fuse_images(image1_path, image2_path, method='average', weights=(0.5, 0.5)):
    
    # Load NIFTI files
    nifti1 = nib.load(image1_path)
    nifti2 = nib.load(image2_path)

    # Check if the image shapes match
    if nifti1.shape != nifti2.shape:
        raise ValueError("The NIFTI images have different shapes and cannot be fused")

    # Get image data as NumPy arrays
    image1 = nifti1.get_fdata()
    image2 = nifti2.get_fdata()
    
    if method == 'average':
        fused_image = (image1 + image2) / 2.0
    elif method == 'weighted':
        weights1, weights2 = weights
        if weights1 + weights2 != 1.0:
            raise ValueError("The weights for the 'weighted' method should sum to 1.0")
        fused_image = weights1 * image1 + weights2 * image2
    else:
        raise ValueError(f"Unknown fusion method: {method}")

    # Normalize the image to the range 0-255
    fused_image = ((fused_image - fused_image.min()) * (255.0 / (fused_image.max() - fused_image.min()))).astype(np.uint8)

    return fused_image
       
        
###
# For the Target channel        
###
# 1-1.Get Affine transform matrix @Nuclear Stain channel @downsized
###

# Fixed and moving images
fixed_image = fixed_image
moving_image = moving_image

# Set the output prefix 
output_prefix = os.path.join(new_directory, "NuclearStainChAffine_")

# Define the ANTs registration command as a list of arguments
ants_registration_command = [
    "antsRegistrationSyN.sh",
    "-d", "3",
    "-f", fixed_image,
    "-m", moving_image,
    "-o", output_prefix,
    "-t", "a",
    "-p", "f"
]

# Set the path for the log file
log_file_path = os.path.join(new_directory, "log_NuclearStainChAffine.txt")

# Run
ants_registration(ants_registration_command, log_file_path)

###
# 1-2.Apply transform generated above to the target channel moving image
###

# Nifti Files of your target
reference_image_downsize = reference_image_downsize
input_image_downsize = input_image_downsize
# Path to the transformation matrix
transformation_matrix_affine = os.path.join(new_directory, "NuclearStainChAffine_0GenericAffine.mat")
# Set the output path
output_image = os.path.join(new_directory, "NuclearStainChAffine_transformed_moving_image.nii.gz")

# Define the antsApplyTransforms command as a list of arguments
ants_apply_transforms_command = [
    "antsApplyTransforms",
    "-d", "3",
    "-i", input_image_downsize,
    "-r", reference_image_downsize,
    "-o", output_image,
    "-t", transformation_matrix_affine,
    "-n", "Linear",
]

# Set the path for the log file
apply_transforms_log_file_path = os.path.join(new_directory, "log_apply_transform_NuclearStainChAffine.txt")

# Run        
ants_apply_transform(
    ants_apply_transforms_command, 
    apply_transforms_log_file_path,
    reference_image=reference_image_downsize,
    input_image=input_image_downsize,
    transformation_matrix=transformation_matrix_affine
)

###
# 1-3.SyN registration using the pre-Affine transformed(above) @ Target channel @Downsize
###

# set the input path
fixed_image_syn = reference_image_downsize
moving_image_syn = output_image

# Set the output prefix 
output_prefix_syn = os.path.join(new_directory, "TargetChSyN_")

# Define the ANTs registration command as a list of arguments
ants_registration_command_syn = [
    "antsRegistrationSyN.sh",
    "-d", "3",
    "-f", fixed_image_syn,
    "-m", moving_image_syn,
    "-o", output_prefix_syn,
    "-t", "s",
    "-p", "f"
]

# Set the path for the log file
log_file_path_syn = os.path.join(new_directory, "log_TargetChSyN.txt")

# Run
ants_registration(ants_registration_command_syn, log_file_path_syn)

###
# 1-4. Apply the two transforms sequentially @ Target channel @fullsize
###

# Nifti Files of your target
reference_image_fullsize = reference_image_fullsize
input_image_fullsize = input_image_fullsize

# Path to the transformation matrix
transformation_matrix_1 = os.path.join(new_directory,"TargetChSyN_0GenericAffine.mat")
transformation_matrix_2 = os.path.join(new_directory,"TargetChSyN_1Warp.nii.gz")

# Set the output path
output_image_syn = os.path.join(new_directory, "TargetCh_fullsize_transformed.nii.gz")

# Define the antsApplyTransforms command as a list of arguments
ants_apply_transforms_command_syn = [
    "antsApplyTransforms",
    "-d", "3",
    "-i", input_image_fullsize,
    "-r", reference_image_fullsize,
    "-o", output_image_syn,
    "-t", transformation_matrix_2,
    "-t", transformation_matrix_1,
    "-t", transformation_matrix_affine,
    "--interpolation", "Linear",
]

# Set the path for the log file
apply_transforms_log_file_path_syn = os.path.join(new_directory, "log_apply_transform_AffineSyN.txt")

# Run
ants_apply_transform(
    ants_apply_transforms_command_syn, 
    apply_transforms_log_file_path_syn,
    reference_image_fullsize,
    input_image_fullsize,
    transformation_matrix_2
)


###
# (Optional but reccomended)
# Resave the output above as 8-bit image; ANTs output is usually in 32-bit
# If you want to save as 16 bit or just save as it is(8bit), please modify params here or simply comment-out this part
###
# Define the 8bit file path
output_image_syn_8bit = os.path.join(new_directory, "TargetCh_fullsize_transformed_8bit.nii.gz")
# Convert to 8bit
Convert8bit(output_image_syn, output_image_syn_8bit)


###
# 1-5.Fuse
###
# Fuse images
fused_image = fuse_images(reference_image_fullsize, output_image_syn_8bit, method='average')

# Save fused image as a new NIFTI file
fused_output_file_path = os.path.join(base_path, f"{os.path.basename(base_path)}_TargetCh_Average_fused_fullsize.nii.gz")
fused_nifti = nib.Nifti1Image(fused_image, affine=nib.load(reference_image_fullsize).affine)
nib.save(fused_nifti, fused_output_file_path)
print('Fused image saved correctly to : ', fused_output_file_path)


###
# For the Nuclear Stain channel
###
# 2-1.Get SyN transform matrix @Nuclear Stain channel @downsized
###
# Fixed and moving images
fixed_image = fixed_image
moving_image = moving_image

# Set the output prefix 
output_prefix_syn_ns = os.path.join(new_path, "NuclearStainChSyN_")

# Define the ANTs registration command as a list of arguments
ants_command_syn = [
    "antsRegistrationSyN.sh",
    "-d", "3",
    "-f", fixed_image,
    "-m", moving_image,
    "-o", output_prefix_syn_ns,
    "-t", "s",
    "-p", "f"
]

# Set the path for the log file
log_path_syn = os.path.join(new_path, "log_NuclearStainChSyN.txt")

# Run
ants_registration(ants_command_syn, log_path_syn)

###
# 2-2.Apply the transform @ Nuclear Stain channel @fullsize
###
# Nifti Files of your target
reference_fullsize = reference_fullsize
input_fullsize = input_fullsize

# Path to the transformation matrix
transform_matrix_1 = os.path.join(new_path,"NuclearStainChSyN_0GenericAffine.mat")
transform_matrix_2 = os.path.join(new_path,"NuclearStainChSyN_1Warp.nii.gz")

# Set the output path
output_image_syn_ns = os.path.join(new_path, "NuclearStainCh_fullsize_transformed.nii.gz")

# Define the antsApplyTransforms command as a list of arguments
ants_apply_transforms_cmd_syn = [
    "antsApplyTransforms",
    "-d", "3",
    "-i", input_fullsize,
    "-r", reference_fullsize,
    "-o", output_image_syn_ns,
    "-t", transform_matrix_2,
    "-t", transform_matrix_1,
    "--interpolation", "Linear",
]

# Set the path for the log file
apply_transforms_log_path_syn = os.path.join(new_path, "log_apply_transform_SyN.txt")

# Run
ants_apply_transform(
    ants_apply_transforms_cmd_syn, 
    apply_transforms_log_path_syn,
    reference_fullsize,
    input_fullsize,
    transform_matrix_2
)


###
# Resave the output above as 8-bit image; ANTs output is usually in 32-bit
# If you want to save as 16 bit or just save as it is(8bit), please modify params here or simply comment-out this part
###

# Define the 8bit file path
output_image_8bit = os.path.join(new_path, "NuclearStainCh_fullsize_transformed_8bit.nii.gz")
# Convert to 8bit
Convert8bit(output_image_syn_ns, output_image_8bit)


###
# 2-3.Fuse
###
# Fuse images
fused_image_ns = fuse_images(reference_fullsize, output_image_8bit, method='average')
# Save fused image as a new NIFTI file
fused_output_path = os.path.join(base_path, f"{os.path.basename(base_path)}_NuclearStainCh_Average_fused_fullsize.nii.gz")
fused_nifti_ns = nib.Nifti1Image(fused_image_ns, affine=nib.load(reference_fullsize).affine)
nib.save(fused_nifti_ns, fused_output_path)
print('Fused iamge saved correctly to : ', fused_output_path)
